In [ ]:
#!/usr/bin/env python


import xarray as xr
import dask
import dask.threaded
import dask.multiprocessing
from dask.distributed import Client
import numpy as np                                                                                        
import zarr



In [19]:
c = Client()

c

Client Scheduler: tcp://127.0.0.1:44742 Dashboard: http://127.0.0.1:40297/status,Cluster Workers: 8 Cores: 48 Memory: 134.91 GB


In [20]:
import sys, glob
import numpy as np
import xarray as xr
import xscale.spectral.fft as xfft
import xscale 
import Wavenum_freq_spec_func as wfs
import time



In [21]:
%time

©zarrv='/store/albert7a/eNATL60/zarr/eNATL60-BLB002-SSV-1h'
dsu=xr.open_zarr(zarru)
dsv=xr.open_zarr(zarrv)




CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 14.5 µs


distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)


In [22]:
dsu

<xarray.Dataset>
Dimensions:              (axis_nbounds: 2, time_counter: 8760, x: 8354, y: 4729)
Coordinates:
    nav_lat              (y, x) float32 dask.array<shape=(4729, 8354), chunksize=(296, 1045)>
    nav_lon              (y, x) float32 dask.array<shape=(4729, 8354), chunksize=(296, 1045)>
    time_centered        (time_counter) datetime64[ns] dask.array<shape=(8760,), chunksize=(744,)>
  * time_counter         (time_counter) datetime64[ns] 2009-07-01T00:30:00 ... 2010-06-30T23:30:00
Dimensions without coordinates: axis_nbounds, x, y
Data variables:
    sozocrtx             (time_counter, y, x) float32 dask.array<shape=(8760, 4729, 8354), chunksize=(24, 120, 120)>
    time_counter_bounds  (time_counter, axis_nbounds) datetime64[ns] dask.array<shape=(8760, 2), chunksize=(744, 2)>
Attributes:
    Conventions:  CF-1.6
    NCO:          4.4.6
    TimeStamp:    08/01/2019 09:34:23 +0100
    description:  ocean U grid variables
    file_name:    eNATL60-BLBT02_1h_20090630_20090704_gr

In [23]:
%%time
lat=dsu['nav_lat']
lon=dsu['nav_lon']
 
latmin = 40.0; latmax = 45.0;
lonmin = -40.0; lonmax = -35.0;

domain = (lonmin<lon) * (lon<lonmax) * (latmin<lat) * (lat<latmax)
where = np.where(domain)

#get indice
jmin = np.min(where[0][:])
jmax = np.max(where[0][:])
imin = np.min(where[1][:])
imax = np.max(where[1][:])

latbox=lat[jmin:jmax,imin:imax]
lonbox=lon[jmin:jmax,imin:imax]


CPU times: user 2.32 s, sys: 408 ms, total: 2.73 s
Wall time: 7.04 s


In [24]:
print(jmin,jmax,imin,imax)

2233 2634 3478 3779


In [25]:
%%time

print('Select dates')
u_JFM=dsu.sel(time_counter=slice('2010-01-01','2010-03-31'))['sozocrtx']
v_JFM=dsv.sel(time_counter=slice('2010-01-01','2010-03-31'))['somecrty']


print('Select box area')
u_JFM_box=u_JFM[:,jmin:jmax,imin:imax].chunk({'time_counter':10,'x':120,'y':120})
v_JFM_box=v_JFM[:,jmin:jmax,imin:imax].chunk({'time_counter':10,'x':120,'y':120})



# - get dx and dy
print('get dx and dy')
dx_JFM,dy_JFM = wfs.get_dx_dy(u_JFM_box[0],lonbox,latbox)


#... Detrend data in all dimension ...
print('Detrend data in all dimension')
u_JFM = wfs.detrendn(u_JFM_box,axes=[0,1,2])
v_JFM = wfs.detrendn(v_JFM_box,axes=[0,1,2])

#... Apply hanning windowing ...') 
print('Apply hanning windowing')
u_JFM = wfs.apply_window(u_JFM, u_JFM.dims, window_type='hanning')
v_JFM = wfs.apply_window(v_JFM, v_JFM.dims, window_type='hanning')


# - get derivatives
derivatives_JFM = wfs.velocity_derivatives(u_JFM, v_JFM, xdim='x', ydim='y', dx={'x': dx_JFM, 'y': dy_JFM})
dudx_JFM = derivatives_JFM['u_x']; dudy_JFM = derivatives_JFM['u_y']
dvdx_JFM = derivatives_JFM['v_x']; dvdy_JFM = derivatives_JFM['v_y']

# - compute terms
phi1_JFM = u_JFM*dudx_JFM + v_JFM*dudy_JFM
phi2_JFM = u_JFM*dvdx_JFM + v_JFM*dvdy_JFM

u_JFMhat = xfft.fft(u_JFM, dim=('time_counter', 'x', 'y'), dx={'x': dx_JFM, 'y': dx_JFM}, sym=True)
v_JFMhat = xfft.fft(v_JFM, dim=('time_counter', 'x', 'y'), dx={'x': dx_JFM, 'y': dx_JFM}, sym=True)

phi1_JFM_hat = xfft.fft(phi1_JFM, dim=('time_counter', 'x', 'y'), dx={'x': dx_JFM, 'y': dx_JFM}, sym=True)
phi2_JFM_hat = xfft.fft(phi2_JFM, dim=('time_counter', 'x', 'y'), dx={'x': dx_JFM, 'y': dx_JFM}, sym=True)

tm1_JFM = (u_JFMhat.conj())*phi1_JFM_hat
tm2_JFM = (v_JFMhat.conj())*phi2_JFM_hat

# - computer transfer
Nk_JFM,Nj_JFM,Ni_JFM = u_JFM.shape
transfer_2D_JFM = -1.0*(tm1_JFM + tm2_JFM)/np.square(Ni_JFM*Nj_JFM)
transfer_term_JFM = transfer_2D_JFM.real

#... Get frequency and wavenumber ... 
print('Get frequency and wavenumber')
ffrequency_JFM = u_JFMhat.f_time_counter
kx_JFM = u_JFMhat.f_x
ky_JFM = u_JFMhat.f_y

#... Get istropic wavenumber ... 
print('Get istropic wavenumber')
wavenumber_JFM,kradial_JFM = wfs.get_wavnum_kradial(kx_JFM,ky_JFM)



Select dates


distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)


Select box area
get dx and dy
Detrend data in all dimension
Apply hanning windowing
Get frequency and wavenumber
Get istropic wavenumber
CPU times: user 2min 22s, sys: 56.5 s, total: 3min 18s
Wall time: 1min 25s


In [26]:
#... Get numpy array ... 
print('Get numpy array')
var_psd_np_JFM = transfer_term_JFM.values


Get numpy array


distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:53228 remote=tcp://127.0.0.1:44742>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:53230 remote=tcp://127.0.0.1:44742>
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:53244 remote=tcp://127.0.0.1:44742>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:53232 remote=tcp://127.0.0.1:44742>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:53242 remote=tcp://127.0.0.1:44742>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tc

ValueError: Internal error occurred

In [ ]:

#... Get 2D frequency-wavenumber field ... 
print('Get transfer')
transfer_JFM = wfs.get_f_k_in_2D(kradial_JFM,wavenumber_JFM,var_psd_np_JFM) 

print('Get flux')
flux_JFM = wfs.get_flux_in_1D(kradial_JFM,wavenumber_JFM,var_psd_np_JFM)
